# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [3]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [4]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.019349955383795, 10.0)

In [6]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000):
    U = np.random.uniform(size = n_sample)
    default_time = -(1/def_rate) * np.log(U)
    terminal_time = np.minimum(default_time, mat)
    isdefault = (default_time <= mat)
    bond_price = (1 - (1-recovery)*isdefault) * np.exp(-rf_rate*terminal_time)
    return bond_price.mean()

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

# Find the mean and std by calling the function 100 times. 

corp_bond_list = [corp_bond(mat, def_rate, rf_rate, recovery, n_sample) for i in range(100)]
np.mean(corp_bond_list), np.std(corp_bond_list)


(0.44032439587236205, 0.002144107443519688)

### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [9]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample//2)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999998, 10.0)

In [10]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    if(antithetic):
        U = np.random.uniform(size = n_sample // 2)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        if (mean_match):
            default_time += 1/def_rate-default_time.mean()
    elif(mean_match):
        U = np.random.uniform(size = n_sample)
        default_time = -(1/def_rate) * np.log(U)
        default_time += 1/def_rate-default_time.mean()
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        
    terminal_time = np.minimum(default_time, mat)
    isdefault = (default_time <= mat)
    bond_price = (1 - (1-recovery)*isdefault) * np.exp(-rf_rate*terminal_time)
    return bond_price.mean()

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

# (i) antithetic
corp_bond_cv_list1 = [corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=False) for i in range(100)]
# (ii) mean_match
corp_bond_cv_list2 = [corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=True) for i in range(100)]
# (iii) both
corp_bond_cv_list3 = [corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=True) for i in range(100)]
print(np.mean(corp_bond_cv_list1), np.std(corp_bond_cv_list1)) 
print(np.mean(corp_bond_cv_list2), np.std(corp_bond_cv_list2))
print(np.mean(corp_bond_cv_list3), np.std(corp_bond_cv_list3))

0.4404289882603583 0.001627494551202447
0.4407263194190134 0.0014568967563575255
0.4405184249954597 0.0015892181666201598


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [12]:
### Put the analytic expression for the corporate bond price

analy_price = (recovery*def_rate) / (def_rate+rf_rate) * (1-np.exp(-(def_rate+rf_rate)*mat)) + np.exp(-(rf_rate+def_rate)*mat)
analy_price

0.44040907156462505

$$
analy\_price = \int_{0}^{T}recovery \times e^{-rt} \lambda e^{-\lambda t}dt + e^{-(r+\lambda)T}
= \frac{recovery \times \lambda}{\lambda+r} [1-e^{-(\lambda+r)T}] + e^{-(r+\lambda)T} 
$$